# Sistema criptográfico RSA (Rivest, Shamir y Adleman).
## Autor: Ricardo Andrés Calvo Méndez

Este sistema criptográfico de clave pública es el más utilizado en el proceso de firmado digital, se caracteríza por ultiliza el problema de factorización de números enteros muy grandes como principal medida de seguridad. Cuando se desea el envío de un mensaje, el emisor debe buscar la clave pública para cifrar con esta el mensaje y posterior a esto el receptor desencripta el mensaje haciendo uso de la clave privada, la cual no debe compartirse por ningún motivo para preservar la seguridad.

El paradigma de la computación cuántica ha estado en proceso de desarrollo durante los ultimos años, pero actualmente ya se cuantan con algoritmos que pueden factorizar en un tiempo razonable los grandes números de la clave publica y privada. Se espera que en los próximos años existan más algoritmos cuánticos que solucionen el problema convirtiendo el sistema RSA en un sistema obsoleto.

***


### ¿Cómo implementar este sistema de encriptación?

***

#### Generación de claves pública y privada.

Para la generación de la clave publica y privada se siguen los siguientes pasos:

1. Se elijen dos números primos $p$ y $q$ diferentes, entre más grandes sean los dos números mas dificil será el proceso de factorización.
2. Se calcula en número $n$ como el producto de los números $p$ y $q$. ($n = pq$)
3. Se calcula el valor de la función $\phi(n)$ de Euler.
4. Se escoje un número $e \in \mathbb{Z}^{+}$ menor que $\phi(n)$ que sea primo relativo con este ($MCD(e, \phi(n)) = 1$).
5. Finalmente se determina un número $d$ que satisfaga $e \cdot d \equiv 1 \mod{\phi(n)}$

La clave pública corresponde a la dupla $(n,e)$ donde $n$ es el modulo y $e$ es el exponente de cifrado, por otra parte la clave privada es la dupla $(n,d)$ donde ahora $d$ es el exponente de descifrado. Ahora se tienen dos procedimientos, la encriptacion y la desencriptación los cuales solo se puden hacer con el conocimiento de tanto la clave pública como de la clave privada.

***

#### Encriptación

Para el cifrado se requiere el uso de la clave pública, esta debe ser enviada por el receptor al emisor, la clave privada debe ser guardada en total secreto por el receptor. Suponiendo que el mensaje a enviar es $m$ con $m < n$ y $MCD(m, n) = 1$, el emisor debe calcular un valor $c$ que cumpla lo siguiente:

<center>$c\equiv m^{e}\mod{n}$</center>

Este será el mensaje que el emisor le envía al receptor.

***

#### Desencriptación

Finalmente el receptor puede recuperar el mensaje $m$ usando la clave privada en la siguiente operación.

<center>$m \equiv c^{d}\mod{n}$</center>

De esta manerá el mensaje fue correctamente desencriptado.

***

### ¿Por qué funciona?

***

***

### Ejemplo e implementación del algorítmo

***